
| <h1> **Hands-on Activity 7.2** </h1> | <h1> **Webscraping using BeautifulSoup and Requests** </h1> |
|--- | --- |
Name: | Cabrera, Gabriel A.<br>
Course and Section: |  CPE 311-CPE22S3<br>
Instructor: | Eng. Neal Barton James Matira
Date Performed: | February 26, 2026
Date Submitted: | February 26, 2026


<hr>


In [1]:
import cv2
#from google.colab.patches import cv2_imshow

webcam = cv2.VideoCapture(0)

while True:
    try:
        check, frame = webcam.read()
        print(check)  # prints True as long as the webcam is running
        print(frame)  # prints matrix values of each frame

        cv2.imshow("Capturing", frame)
        key = cv2.waitKey(1)

        if key == ord('s'):
            cv2.imwrite('saved_img.jpg', frame)
            webcam.release()

            img_new = cv2.imread('saved_img.jpg', cv2.IMREAD_GRAYSCALE)
            cv2.imshow("Captured Image", img_new)
            cv2.waitKey(1650)
            cv2.destroyAllWindows()

            print("Processing image...")
            img_ = cv2.imread('saved_img.jpg', cv2.IMREAD_ANYCOLOR)

            print("Converting RGB image to grayscale...")
            gray = cv2.cvtColor(img_, cv2.COLOR_BGR2GRAY)
            print("Converted RGB image to grayscale...")

            print("Resizing image to 28x28 scale...")
            img_ = cv2.resize(gray, (28, 28))
            print("Resized...")

            cv2.imwrite('saved_img-final.jpg', img_)
            print("Image saved!")
            break

        elif key == ord('q'):
            print("Turning off camera.")
            webcam.release()
            print("Camera off.")
            print("Program ended.")
            cv2.destroyAllWindows()
            break

    except KeyboardInterrupt:
        print("Turning off camera.")
        webcam.release()
        print("Camera off.")
        print("Program ended.")
        cv2.destroyAllWindows()
        break

ModuleNotFoundError: No module named 'cv2'

In [2]:
!pip install sounddevice 

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/365.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/365.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/365.3 kB ? eta -:--:--
   ------------------------------------ --- 337.9/365.3 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 365.3/365.3 kB 2.3 MB/s eta 0:00:00


In [3]:
!pip install wavio

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip3 install scipy 

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!apt-get install libportaudio2

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
# import required libraries 
import sounddevice as sd 
from scipy.io.wavfile import write 
import wavio as wv 
  
# Sampling frequency 
freq = 44100 
  
# Recording duration 
duration = 5 
  
# Start recorder with the given values  
# of duration and sample frequency 
recording = sd.rec(int(duration * freq), 
 samplerate=freq, channels=2) 
  
# Record audio for the given number of seconds 
sd.wait() 
  
# This will convert the NumPy array to an audio 
# file with the given sampling frequency 
write("recording0.wav", freq, recording) 
 
# Convert the NumPy array to audio file 
wv.write("recording1.wav", recording, freq, sampwidth=2) 


PortAudioError: Error querying device -1

In [7]:
!pip install bs4 

Defaulting to user installation because normal site-packages is not writeable


In [8]:
pip install requests 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests 
from bs4 import BeautifulSoup 
  
def getdata(url): 
 r = requests.get(url) 
 return r.text 
  
htmldata = getdata("https://www.google.com/") 
soup = BeautifulSoup(htmldata, 'html.parser') 
for item in soup.find_all('img'): 
 print(item['src']) 


/images/branding/googlelogo/1x/googlelogo_white_background_color_272x92dp.png


In [10]:
!pip install selenium
!pip install webdriver-manager
!pip install pillow
!pip install requests

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import time, os, io, requests
from PIL import Image

# Setup Firefox
local_driver_path = "./geckodriver.exe"
firefox_binary_path = r"C:\Program Files\Mozilla Firefox\firefox.exe"

options = Options()
options.binary_location = firefox_binary_path
options.headless = True
options.set_preference("general.useragent.override",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/118.0.5993.118 Safari/537.36"
)

service = Service(executable_path=local_driver_path)
driver = webdriver.Firefox(service=service, options=options)

# Scroll function
def scroll_to_end(driver, scrolls=3):
    for _ in range(scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

# Get image URLs
def get_image_urls(query, total_images):
    driver.get(f"https://www.google.com/search?q={query}&tbm=isch")
    image_urls = set()
    results_start = 0

    while len(image_urls) < total_images:
        scroll_to_end(driver, scrolls=2)
        thumbnails = driver.find_elements(By.CSS_SELECTOR, "img[src^='https://']")
        print(f"Found {len(thumbnails)} thumbnails")

        for img in thumbnails[results_start:]:
            try:
                img.click()
                time.sleep(1)
                images = driver.find_elements(By.CSS_SELECTOR, "img[src^='https://']")
                for image in images:
                    src = image.get_attribute("src")
                    if src and "http" in src:
                        image_urls.add(src)
                if len(image_urls) >= total_images:
                    break
            except:
                continue
        results_start = len(thumbnails)

    return list(image_urls)[:total_images]

# Download images
def download_images(folder, urls):
    if not os.path.exists(folder):
        os.makedirs(folder)
    for i, url in enumerate(urls):
        try:
            image_content = requests.get(url, timeout=5).content
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file).convert("RGB")
            file_path = os.path.join(folder, f"{i+1}.jpg")
            image.save(file_path, "JPEG", quality=85)
            print(f"Saved: {file_path}")
        except Exception as e:
            print(f"Could not save {url} - {e}")

# MAIN
search_term = "Circle"
total_images = 5
save_folder = r"C:\Users\tipqc\Desktop\CPE-311-Github-Repository-for-Data-Science-1"

urls = get_image_urls(search_term, total_images)
download_images(save_folder, urls)

driver.quit()
print("Done!")

Found 457 thumbnails
Saved: C:\Users\tipqc\Desktop\CPE-311-Github-Repository-for-Data-Science-1\1.jpg
Saved: C:\Users\tipqc\Desktop\CPE-311-Github-Repository-for-Data-Science-1\2.jpg


c:\ProgramData\anaconda3\Lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Saved: C:\Users\tipqc\Desktop\CPE-311-Github-Repository-for-Data-Science-1\3.jpg
Saved: C:\Users\tipqc\Desktop\CPE-311-Github-Repository-for-Data-Science-1\4.jpg
Saved: C:\Users\tipqc\Desktop\CPE-311-Github-Repository-for-Data-Science-1\5.jpg
Done!


In [5]:
from requests import get 
headers = {    
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0 win64; x64)"
}

url = 'https://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1' 
response = get(url) 
print(response.text[:500])

In [3]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
import time
from bs4 import BeautifulSoup

# setup firefox
local_driver_path = "./geckodriver.exe"
firefox_binary_path = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options = Options()
options.binary_location =  firefox_binary_path
#service = Service(GeckoDriverManager().install())
service = Service(executable_path=local_driver_path)
#driver = webdriver.Firefox(service=service)
driver = webdriver.Firefox(service=service, options=options)

# test
html = driver.get("https://www.google.com")
print ("Page Title:", driver.title)

Page Title: Google


In [6]:
from requests import get 
headers = {    
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0 win64; x64)"
}

url = 'https://www.imdb.com/search/title/?release_date=2017-01-01,2017-12-31&sort=num_votes,desc' 
response = get(url) 
driver.get(url)
print(response.text[:500])

In [7]:
driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
html = driver.page_source

In [8]:
soup = BeautifulSoup(html,'html.parser')

In [9]:
data = soup.find(class_='ipc-metadata-list ipc-metadata-list--dividers-between sc-d24d5d37-0 hDHQeM detailed-list-view ipc-metadata-list--base')

In [10]:
movie_containers = soup.find_all('div', class_ = 'ipc-metadata-list-summary-item__c')
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


In [11]:
first_movie = movie_containers[0]
first_movie

<div class="ipc-metadata-list-summary-item__c"><div class="ipc-metadata-list-summary-item__tc"><span aria-disabled="false" class="ipc-metadata-list-summary-item__t ipc-btn--not-interactable"></span><div class="sc-fc35a1ef-1 lmHCrT dli-parent"><div class="sc-fc35a1ef-0 hTMtRz"><div class="sc-d0224b4e-0 jfogmY dli-poster-container"><div class="ipc-poster ipc-poster--base ipc-poster--media-radius ipc-poster--wl-true ipc-poster--dynamic-width ipc-sub-grid-item ipc-sub-grid-item--span-2" role="group"><div class="ipc-media ipc-media--poster-27x40 ipc-image-media-ratio--poster-27x40 ipc-media--media-radius ipc-media--base ipc-media--poster-s ipc-poster__poster-image ipc-media__img" style="width:100%"><img alt="Hugh Jackman in Logan (2017)" class="ipc-image" loading="lazy" sizes="50vw, (min-width: 480px) 34vw, (min-width: 600px) 26vw, (min-width: 1024px) 16vw, (min-width: 1280px) 16vw" src="https://m.media-amazon.com/images/M/MV5BM2JjODdkMGMtNmY2YS00OGM2LThiY2YtZGYyNzE4Nzc2ODA0XkEyXkFqcGc@._V1

In [12]:
first_movie.div

<div class="ipc-metadata-list-summary-item__tc"><span aria-disabled="false" class="ipc-metadata-list-summary-item__t ipc-btn--not-interactable"></span><div class="sc-fc35a1ef-1 lmHCrT dli-parent"><div class="sc-fc35a1ef-0 hTMtRz"><div class="sc-d0224b4e-0 jfogmY dli-poster-container"><div class="ipc-poster ipc-poster--base ipc-poster--media-radius ipc-poster--wl-true ipc-poster--dynamic-width ipc-sub-grid-item ipc-sub-grid-item--span-2" role="group"><div class="ipc-media ipc-media--poster-27x40 ipc-image-media-ratio--poster-27x40 ipc-media--media-radius ipc-media--base ipc-media--poster-s ipc-poster__poster-image ipc-media__img" style="width:100%"><img alt="Hugh Jackman in Logan (2017)" class="ipc-image" loading="lazy" sizes="50vw, (min-width: 480px) 34vw, (min-width: 600px) 26vw, (min-width: 1024px) 16vw, (min-width: 1280px) 16vw" src="https://m.media-amazon.com/images/M/MV5BM2JjODdkMGMtNmY2YS00OGM2LThiY2YtZGYyNzE4Nzc2ODA0XkEyXkFqcGc@._V1_QL75_UX90_CR0,1,90,133_.jpg" srcset="https://m

In [13]:
first_movie.a

<a aria-label="View title page for Logan" class="ipc-lockup-overlay ipc-focusable ipc-focusable--constrained" href="/title/tt3315342/?ref_=sr_i_1"><div class="ipc-lockup-overlay__screen"></div></a>

In [90]:
first_movie.h3

<h3 class="ipc-title__text">1. Logan</h3>

In [14]:
first_movie.h3.a

In [15]:
first_name = first_movie.h3.text
first_name

'1. Logan'

In [16]:
first_year = first_movie.find('span', class_ = 'sc-a55f6282-6 iMumIM dli-title-metadata-item')
first_year

<span class="sc-a55f6282-6 iMumIM dli-title-metadata-item">2017</span>

In [17]:
first_rating = soup.find('span', class_='ipc-rating-star--rating')
first_rating.text

'8.1'

### Metascore

In [18]:
first_mscore = first_movie.find('span', class_ = 'sc-9fe7b0ef-0 hDuMnh metacritic-score-box')
first_mscore = int(first_mscore.text)
print(first_mscore)

77


### Number of Votes

In [19]:
first_votes = first_movie.find('span', class_ = 'ipc-rating-star--voteCount')
first_votes

<span class="ipc-rating-star--voteCount"> (<!-- -->919K<!-- -->)</span>

In [20]:
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
# Extract data from individual movie container
for container in movie_containers:
# If the movie has Metascore, then extract:
    if container.find('div', class_ = 'sc-fc35a1ef-1 lmHCrT dli-parent') is not None:
# The name
        name = container.h3.text
        names.append(name.split(".",1)[1])
# The year
        year = container.find('span', class_ = 'sc-a55f6282-6 iMumIM dli-title-metadata-item')
        years.append(year.text.split(")")[0][0:])
# The IMDB rating
        imdb = container.find('span', class_='ipc-rating-star--rating')
        imdb_ratings.append(imdb.text.split(")")[0][0:])
# The Metascore
        if container.find('span', class_ = 'sc-9fe7b0ef-0 hDuMnh metacritic-score-box') is not None:
                m_score = container.find('span', class_ = 'sc-9fe7b0ef-0 hDuMnh metacritic-score-box')
                metascores.append(m_score.text.split(")")[0][0:])
        else:
                m_score = container.find('span', class_ = 'sc-9fe7b0ef-0 hDuMnh metacritic-score-box')
                metascores.append(m_score)
# The number of votes
        vote = container.find('span', class_ = 'ipc-rating-star--voteCount')
        votes.append(vote.text.split(")")[0][2:])

In [21]:
import pandas as pd
test_df = pd.DataFrame({'movie': names,
'year': years,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes
})
print(test_df.info())
test_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movie      50 non-null     object
 1   year       50 non-null     object
 2   imdb       50 non-null     object
 3   metascore  41 non-null     object
 4   votes      50 non-null     object
dtypes: object(5)
memory usage: 2.1+ KB
None


,movie,year,imdb,metascore,votes
0,Logan,2017,8.1,77,919K
1,Thor: Ragnarok,2017,7.9,74,887K
2,Guardians of the Galaxy Vol. 2,2017,7.6,67,822K
3,Get Out,2017,7.8,85,807K
4,Dunkirk,2017,7.8,94,801K
5,Spider-Man: Homecoming,2017,7.4,73,789K
6,Blade Runner 2049,2017,8.0,81,746K
7,Wonder Woman,2017,7.3,76,733K
8,Star Wars: Episode VIII - The Last Jedi,2017,6.8,84,714K
9,Coco,2017,8.4,81,697K


In [190]:
from time import time
from time import sleep
from time import time, sleep
from random import randint
from requests import get
from bs4 import BeautifulSoup
from IPython.core.display import clear_output
from warnings import warn

pages = ['1','2','3','4','5']
years_url = ['2017', '2018', '2019', '2020']

# Lists to store data
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

start_time = time()
requests = 0

for year_url in years_url:
    for page in pages:

        response = get(
            'https://www.imdb.com/search/title?release_date=' 
            + year_url +
            '&sort=num_votes,desc&page=' 
            + page,
            headers=headers
        )

        sleep(randint(8,15))

        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(
            requests, requests/elapsed_time))
        clear_output(wait=True)

        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(
                requests, response.status_code))

        if requests > 72:
            warn('Number of requests was greater than expected.')
            break

        page_html = BeautifulSoup(response.text, 'html.parser')

        mv_containers = page_html.find_all(
            'div', class_='lister-item mode-advanced')

        for container in mv_containers:

            if container.find('div', class_ = 'sc-fc35a1ef-1 lmHCrT dli-parent') is not None:
# The name
                name = container.h3.text
                names.append(name.split(".",1)[1])
# The year
                year = container.find('span', class_ = 'sc-a55f6282-6 iMumIM dli-title-metadata-item')
                years.append(year.text.split(")")[0][0:])
# The IMDB rating
                imdb = container.find('span', class_='ipc-rating-star--rating')
                imdb_ratings.append(imdb.text.split(")")[0][0:])
# The Metascore
                if container.find('span', class_ = 'sc-9fe7b0ef-0 hDuMnh metacritic-score-box') is not None:
                    m_score = container.find('span', class_ = 'sc-9fe7b0ef-0 hDuMnh metacritic-score-box')
                    metascores.append(m_score.text.split(")")[0][0:])
                else:
                    m_score = container.find('span', class_ = 'sc-9fe7b0ef-0 hDuMnh metacritic-score-box')
                    metascores.append(m_score)
# The number of votes
                vote = container.find('span', class_ = 'ipc-rating-star--voteCount')
                votes.append(vote.text.split(")")[0][2:])

KeyboardInterrupt: 

In [59]:
movie_ratings = pd.DataFrame({'movie': names,
'year': years,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes
})
print(movie_ratings.info())
movie_ratings.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movie      50 non-null     object
 1   year       50 non-null     object
 2   imdb       50 non-null     object
 3   metascore  41 non-null     object
 4   votes      50 non-null     object
dtypes: object(5)
memory usage: 2.1+ KB
None


,movie,year,imdb,metascore,votes
0,Logan,2017,8.1,77,919K
1,Thor: Ragnarok,2017,7.9,74,887K
2,Guardians of the Galaxy Vol. 2,2017,7.6,67,822K
3,Get Out,2017,7.8,85,807K
4,Dunkirk,2017,7.8,94,801K
5,Spider-Man: Homecoming,2017,7.4,73,789K
6,Blade Runner 2049,2017,8.0,81,746K
7,Wonder Woman,2017,7.3,76,733K
8,Star Wars: Episode VIII - The Last Jedi,2017,6.8,84,714K
9,Coco,2017,8.4,81,697K


In [23]:
movie_ratings.tail(10)

,movie,year,imdb,metascore,votes
40,King Arthur: Legend of the Sword,2017,6.7,41,246K
41,The End of the F***ing World,2017–2019,8.0,None,244K
42,Big Little Lies,2017–2026,8.4,None,243K
43,Darkest Hour,2017,7.4,75,242K
44,Ghost in the Shell,2017,6.3,52,239K
45,American Made,2017,7.1,65,231K
46,The Mummy,2017,5.4,34,221K
47,Atomic Blonde,2017,6.7,63,219K
48,Baywatch,2017,5.5,37,217K
49,The Killing of a Sacred Deer,2017,7.0,73,213K


In [60]:
movie_ratings.to_csv("Movie_Ratings_7.2")

In [61]:
movie_ratings['year'].unique()

array(['2017', '2017–2021', '2017–2020', '2017–2019', '2017–2022',
       '2017–2025', '2017–2026'], dtype=object)

In [62]:
movie_ratings.dtypes

movie        object
year         object
imdb         object
metascore    object
votes        object
dtype: object

In [63]:
movie_ratings['year'] = (movie_ratings.year.apply(lambda x:x.replace('(I)','')))

In [64]:
movie_ratings['year'].unique()

array(['2017', '2017–2021', '2017–2020', '2017–2019', '2017–2022',
       '2017–2025', '2017–2026'], dtype=object)

In [65]:
movie_ratings['year'] = (movie_ratings.year.apply(lambda x:x.replace('(II)','')))

In [66]:
movie_ratings['year'] = (movie_ratings.year.apply(lambda x:x.replace('(III)','')))

In [67]:
movie_ratings['year'].unique()

array(['2017', '2017–2021', '2017–2020', '2017–2019', '2017–2022',
       '2017–2025', '2017–2026'], dtype=object)

In [68]:
movie_ratings['year'] = (movie_ratings.year.apply(lambda x:x.replace('(','')))

In [69]:
movie_ratings['year'].unique()

array(['2017', '2017–2021', '2017–2020', '2017–2019', '2017–2022',
       '2017–2025', '2017–2026'], dtype=object)

In [70]:
movie_ratings['year'] = (movie_ratings.year.apply(lambda x:x.replace(')','')))

In [71]:
movie_ratings['year'].unique()

array(['2017', '2017–2021', '2017–2020', '2017–2019', '2017–2022',
       '2017–2025', '2017–2026'], dtype=object)

In [73]:
movie_ratings.head(10)

,movie,year,imdb,metascore,votes
0,Logan,2017,8.1,77,919K
1,Thor: Ragnarok,2017,7.9,74,887K
2,Guardians of the Galaxy Vol. 2,2017,7.6,67,822K
3,Get Out,2017,7.8,85,807K
4,Dunkirk,2017,7.8,94,801K
5,Spider-Man: Homecoming,2017,7.4,73,789K
6,Blade Runner 2049,2017,8.0,81,746K
7,Wonder Woman,2017,7.3,76,733K
8,Star Wars: Episode VIII - The Last Jedi,2017,6.8,84,714K
9,Coco,2017,8.4,81,697K


In [74]:
movie_ratings.tail(10)

,movie,year,imdb,metascore,votes
40,King Arthur: Legend of the Sword,2017,6.7,41,246K
41,The End of the F***ing World,2017–2019,8.0,None,244K
42,Big Little Lies,2017–2026,8.4,None,243K
43,Darkest Hour,2017,7.4,75,242K
44,Ghost in the Shell,2017,6.3,52,239K
45,American Made,2017,7.1,65,231K
46,The Mummy,2017,5.4,34,221K
47,Atomic Blonde,2017,6.7,63,219K
48,Baywatch,2017,5.5,37,217K
49,The Killing of a Sacred Deer,2017,7.0,73,213K


In [75]:
movie_ratings


,movie,year,imdb,metascore,votes
0,Logan,2017,8.1,77,919K
1,Thor: Ragnarok,2017,7.9,74,887K
2,Guardians of the Galaxy Vol. 2,2017,7.6,67,822K
3,Get Out,2017,7.8,85,807K
4,Dunkirk,2017,7.8,94,801K
5,Spider-Man: Homecoming,2017,7.4,73,789K
6,Blade Runner 2049,2017,8.0,81,746K
7,Wonder Woman,2017,7.3,76,733K
8,Star Wars: Episode VIII - The Last Jedi,2017,6.8,84,714K
9,Coco,2017,8.4,81,697K
